# <span style="font-width:bold; font-size: 3rem; color:#2656a3;">**Data Engineering and Machine Learning Operations in Business** </span> <span style="font-width:bold; font-size: 3rem; color:#333;">- Part 01: Feature Backfill</span>

## 🗒️ This notebook is divided into the following sections:
1. Load the data and process features
2. Connect to the Hopsworks feature store
3. Create feature groups and upload them to the feature store

## <span style='color:#2656a3'> ⚙️ Import of libraries and packages

First, we'll install the Python packages required for this notebook. We'll use the --quiet command after specifying the names of the libraries to ensure a silent installation process. Then, we'll proceed to import all the necessary libraries.

In [1]:
# Install of the packages for hopsworks
# !pip install -U hopsworks --quiet

In [3]:
# Importing of the packages for the needed libraries for the Jupyter notebook
import pandas as pd
import requests

# Ignore warnings
import warnings 
warnings.filterwarnings('ignore')

## <span style="color:#2656a3;"> 💽 Load the historical data

The data you will use comes from three different sources:

- Electricity prices in Denmark per day from [Energinet](https://www.energidataservice.dk).
- Different meteorological observations from [Open meteo](https://www.open-meteo.com).
- Danish Calendar with the type if the date is a national holiday or not. This files is made manually by the group and is located in the "*data*" folder inside this repository.

### <span style="color:#2656a3;">💸 Electricity prices per day from Energinet
*Hvis vi skal have tariffer med i modellen, anbefales det at vi sætter en faktor på 0.2 i tidsrummet 22 - 16 og en faktor på 0.6 eller 0.7 i tidsrummet 17 - 21.*

This first dataset is Electricity prices per day from Energinet/Dataservice. Here we use 

In [4]:
# Defining the URL for the API call to the electricity price data
electricity_api_url = ('https://api.energidataservice.dk/dataset/Elspotprices?offset=0&start=2022-01-01T00:00&end=2023-12-31T23:59&filter=%7B%22PriceArea%22:[%22DK1%22]%7D&sort=HourUTC%20DESC')

In [5]:
# Fetch data from the API and make the output to a pandas dataframe
electricity_data_response = requests.get(electricity_api_url)
electricity_data = electricity_data_response.json()
electricity_df = pd.DataFrame(electricity_data['records'])

# Checking the result of the API call. If the response if 200 then the API call was successfull
print(electricity_data_response)

<Response [200]>


In [6]:
# Display the first 5 rows of the dataframe
electricity_df.head()

,HourUTC,HourDK,PriceArea,SpotPriceDKK,SpotPriceEUR
0,2023-12-31T22:00:00,2023-12-31T23:00:00,DK1,200.309998,26.870001
1,2023-12-31T21:00:00,2023-12-31T22:00:00,DK1,213.729996,28.670000
2,2023-12-31T20:00:00,2023-12-31T21:00:00,DK1,220.660004,29.600000
3,2023-12-31T19:00:00,2023-12-31T20:00:00,DK1,260.100006,34.889999
4,2023-12-31T18:00:00,2023-12-31T19:00:00,DK1,295.510010,39.639999


In [7]:
# Datapreprocessing by making the spotprice per kwh instead of mwh
electricity_df['SpotPriceDKK_KWH'] = electricity_df['SpotPriceDKK'] / 1000

In [8]:
# Datacleaning by removing the columns that are not needed
electricity_df.drop('SpotPriceDKK', axis=1, inplace=True)
electricity_df.drop('SpotPriceEUR', axis=1, inplace=True)
electricity_df.drop('HourUTC', axis=1, inplace=True)

In [9]:
# Renaming the columns and reformating the time column
electricity_df.rename(columns={'HourDK': 'time'}, inplace=True)

In [10]:
# Formatting the date column
electricity_df['time'] = electricity_df['time'].astype(str).str[:-3]
electricity_df['date'] = electricity_df['time'].str[:10]

In [11]:
# Creating a new column for the date called electricity_temporary_date_column and insert it as the first column in the dataframe
electricity_temporary_date_column = electricity_df.pop('date')
electricity_df.insert(0, 'date', electricity_temporary_date_column)

In [12]:
# Convert string 'date' column to date type and 'time' column to datetime format
electricity_df['date'] = pd.to_datetime(electricity_df['date'], format='%Y-%m-%d').dt.date
electricity_df['time'] = pd.to_datetime(electricity_df['time'])

In [13]:
# Display the first 5 rows of the dataframe
electricity_df.head()

,date,time,PriceArea,SpotPriceDKK_KWH
0,2023-12-31,2023-12-31 23:00:00,DK1,0.20031
1,2023-12-31,2023-12-31 22:00:00,DK1,0.21373
2,2023-12-31,2023-12-31 21:00:00,DK1,0.22066
3,2023-12-31,2023-12-31 20:00:00,DK1,0.26010
4,2023-12-31,2023-12-31 19:00:00,DK1,0.29551


In [14]:
# Showing the information for the electricity dataframe
electricity_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17520 entries, 0 to 17519
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   date              17520 non-null  object        
 1   time              17520 non-null  datetime64[ns]
 2   PriceArea         17520 non-null  object        
 3   SpotPriceDKK_KWH  17520 non-null  float64       
dtypes: datetime64[ns](1), float64(1), object(2)
memory usage: 547.6+ KB


In [15]:
# Fetching historical electricity prices data
electricity = electricity_prices.fetch_electricity_prices(historical=True, start='2022-01-01', end='2023-12-31')
electricity = electricity[(electricity['PriceArea'] == "DK1")]
electricity.head()

,date,time,PriceArea,SpotPriceDKK_KWH
0,2023-12-31,2023-12-31 23:00:00,DK1,0.20031
2,2023-12-31,2023-12-31 22:00:00,DK1,0.21373
4,2023-12-31,2023-12-31 21:00:00,DK1,0.22066
6,2023-12-31,2023-12-31 20:00:00,DK1,0.26010
8,2023-12-31,2023-12-31 19:00:00,DK1,0.29551


### <span style="color:#2656a3;"> 🌤 Weather measurements from Open Meteo

Burde have enddate 2023-12-31. url = ("https://archive-api.open-meteo.com/v1/archive?latitude=57.048&longitude=9.9187&start_date=2022-01-01&end_date=2023-12-31&hourly=temperature_2m,relative_humidity_2m,precipitation,rain,snowfall,weather_code,cloud_cover,wind_speed_10m,wind_gusts_10m")

In [16]:
# Defining the URL for the API call to the weather data   
weather_api_url = ("https://archive-api.open-meteo.com/v1/archive?latitude=57.048&longitude=9.9187&start_date=2022-01-01&end_date=2023-12-31&hourly=temperature_2m,relative_humidity_2m,precipitation,rain,snowfall,weather_code,cloud_cover,wind_speed_10m,wind_gusts_10m")

In [17]:
# Fetch data from the API and make the output to a pandas dataframe
weather_data_response = requests.get(weather_api_url)
weather_data = weather_data_response.json()
weather_df = pd.DataFrame(weather_data['hourly'])

# Checking the result of the API call. If the response if 200 then the API call was successfull
print(weather_data_response)

<Response [200]>


In [18]:
# Formatting the date column
weather_df['date'] = weather_df['time'].str[:10]

In [19]:
# Creating a new column for the date called weather_temporary_date_column and insert it as the first column in the dataframe
weather_temporary_date_column = weather_df.pop('date')
weather_df.insert(0, 'date', weather_temporary_date_column)

In [20]:
# Convert string 'date' column to date type and 'time' column to datetime format
weather_df['date'] = pd.to_datetime(weather_df['date'], format='%Y-%m-%d').dt.date
weather_df['time'] = pd.to_datetime(weather_df['time'])

In [21]:
# Display the first 5 rows of the dataframe
weather_df.head()

,date,time,temperature_2m,relative_humidity_2m,precipitation,rain,snowfall,weather_code,cloud_cover,wind_speed_10m,wind_gusts_10m
0,2022-01-01,2022-01-01 00:00:00,6.7,100,0.0,0.0,0.0,3,100,16.2,36.0
1,2022-01-01,2022-01-01 01:00:00,6.6,100,0.0,0.0,0.0,3,100,16.2,30.2
2,2022-01-01,2022-01-01 02:00:00,6.7,99,0.0,0.0,0.0,3,100,15.5,30.6
3,2022-01-01,2022-01-01 03:00:00,6.7,100,0.0,0.0,0.0,3,100,12.7,28.8
4,2022-01-01,2022-01-01 04:00:00,6.7,99,0.0,0.0,0.0,3,100,10.6,23.8


In [22]:
# Showing the information for the weather dataframe
weather_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17520 entries, 0 to 17519
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   date                  17520 non-null  object        
 1   time                  17520 non-null  datetime64[ns]
 2   temperature_2m        17520 non-null  float64       
 3   relative_humidity_2m  17520 non-null  int64         
 4   precipitation         17520 non-null  float64       
 5   rain                  17520 non-null  float64       
 6   snowfall              17520 non-null  float64       
 7   weather_code          17520 non-null  int64         
 8   cloud_cover           17520 non-null  int64         
 9   wind_speed_10m        17520 non-null  float64       
 10  wind_gusts_10m        17520 non-null  float64       
dtypes: datetime64[ns](1), float64(6), int64(3), object(1)
memory usage: 1.5+ MB


In [23]:
# Fetching historical electricity prices data
weater = weater_measures.fetch_weater_measures()
weater.head()

,date,time,temperature_2m,relative_humidity_2m,precipitation,rain,snowfall,weather_code,cloud_cover,wind_speed_10m,wind_gusts_10m
0,2022-01-01,2022-01-01 00:00:00,6.7,100,0.0,0.0,0.0,3,100,16.2,36.0
1,2022-01-01,2022-01-01 01:00:00,6.6,100,0.0,0.0,0.0,3,100,16.2,30.2
2,2022-01-01,2022-01-01 02:00:00,6.7,99,0.0,0.0,0.0,3,100,15.5,30.6
3,2022-01-01,2022-01-01 03:00:00,6.7,100,0.0,0.0,0.0,3,100,12.7,28.8
4,2022-01-01,2022-01-01 04:00:00,6.7,99,0.0,0.0,0.0,3,100,10.6,23.8


### <span style="color:#2656a3;"> 🗓️ Calendar of Danish workdays and holidays 

In [24]:
# Read csv file with calender
calender_df = pd.read_csv('https://raw.githubusercontent.com/Camillahannesbo/MLOPs-Assignment-/main/data/calendar_incl_holiday.csv', delimiter=';', usecols=['date', 'type'])
 
# Display the DataFrame
calender_df

,date,type
0,01/01/2022,Not a Workday
1,02/01/2022,Not a Workday
2,03/01/2022,Workday
3,04/01/2022,Workday
4,05/01/2022,Workday
...,...,...
1091,27/12/2024,Workday
1092,28/12/2024,Not a Workday
1093,29/12/2024,Not a Workday
1094,30/12/2024,Workday


In [25]:
# Formatting the date column by replacing the / with -
calender_df['date'] = calender_df['date'].str.replace('/', '-')

In [26]:
# Defining the function for flipping the date to the left

# Flip the date to the left
def flip_date_left(date):
    parts = date.split("-")  # Assuming the date format is "YYYY-MM-DD"
    flipped_date = "-".join(parts[::-1])
    return flipped_date

# Flip the date to the left for the date column
def flip_dates_left_in_column(column):
    flipped_column = [flip_date_left(date) for date in column]
    return flipped_column

In [27]:
# Make the new flipped_dates_column based on the function above and insert it as the 'date' column in the dataframe
flipped_dates_column = flip_dates_left_in_column(calender_df['date'])
calender_df['date'] = flipped_dates_column

In [28]:
# Convert string 'date' column to date type
calender_df['date'] = pd.to_datetime(calender_df['date'], format='%Y-%m-%d').dt.date

In [29]:
# Display the first 5 rows of the dataframe
calender_df.head()

,date,type
0,2022-01-01,Not a Workday
1,2022-01-02,Not a Workday
2,2022-01-03,Workday
3,2022-01-04,Workday
4,2022-01-05,Workday


In [30]:
# Showing the information for the calender dataframe
calender_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1096 entries, 0 to 1095
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   date    1096 non-null   object
 1   type    1096 non-null   object
dtypes: object(2)
memory usage: 17.3+ KB


## <span style="color:#2656a3;"> 📡 Connecting to Hopsworks Feature Store

First we will connect to Hopsworks Feature Store so we can access and create Feature Groups.
Feature groups can also be used to define a namespace for features. For instance, in a real-life setting you would likely want to experiment with different window lengths. In that case, you can create feature groups with identical schema for each window length. 

Before you can create a feature group you need to connect to our feature store.

In [ ]:
import hopsworks

project = hopsworks.login()

fs = project.get_feature_store()

### <span style="color:#2656a3;"> 🪄 Creating Feature Groups

When creating a feature group, you must name it and designate a primary key. Additionally, it's helpful to include a description of the feature group's contents and a version number; if not defined, it will default to `1`. 

We've configured `online_enabled` as `True` to enable the feature group to be read via the Online API for a Feature View.

In [ ]:
# Creating the feature group for the weater data
weather_fg = fs.get_or_create_feature_group(
    name="weather_measurements",
    version=1,
    description="Weather measurements from Open Meteo API",
    primary_key=["date"],
    event_time="time",
    online_enabled=True,
)

By now, you've only outlined metadata for the feature group. There's no data stored, nor is there a defined schema for it. To establish persistence for the feature group, you'll need to populate it with its associated data using the `insert` function

In [ ]:
# Inserting the weather_df into the feature group named weather_fg
weather_fg.insert(weather_df)

We make a descriptions for each feature we put into the feature group. In this way we are adding more information and documentation to the user

In [ ]:
weather_feature_descriptions = [
    {"name": "date", "description": "Date of the weather measurement"},
    {"name": "time", "description": "Time of the weather measurement"},
    {"name": "temperature_2m", "description": "Temperature at 2m above ground"},
    {"name": "relative_humidity_2m", "description": "Relative humidity at 2m above ground"},
    {"name": "precipitation", "description": "Precipitation"},
    {"name": "rain", "description": "Rain"},
    {"name": "snowfall", "description": "Snowfall"},   
    {"name": "weather_code", "description": "Weather code"},   
    {"name": "cloud_cover", "description": "Cloud cover"},   
    {"name": "wind_speed_10m", "description": "Wind speed at 10m above ground"},   
    {"name": "wind_gusts_10m", "description": "Wind gusts at 10m above ground"},   
]

for desc in weather_feature_descriptions: 
    weather_fg.update_feature_description(desc["name"], desc["description"])

We replicate the process for both the `electricity_fg` and `danish_holidays_fg` by establishing feature groups and inserting the dataframes into their respective feature groups.

In [ ]:
# Creating the feature group for the electricity prices
electricity_fg = fs.get_or_create_feature_group(
    name="electricity_prices",
    version=1,
    description="Electricity prices from Energidata API",
    primary_key=["date"],
    online_enabled=True,
    event_time="time",
)

In [ ]:
# Inserting the electricity_df into the feature group named electricity_fg
electricity_fg.insert(electricity_df)

In [ ]:
electricity_feature_descriptions = [
    {"name": "date", "description": "Date of the electricity measurement"},
    {"name": "time", "description": "Time of the electricity measurement"},
    {"name": "PriceArea", "description": "Price area for the electricity measurement"},
    {"name": "SpotPriceDKK_KWH", "description": "Spot price in DKK per KWH"}, 
]

for desc in electricity_feature_descriptions: 
    electricity_fg.update_feature_description(desc["name"], desc["description"])

In [ ]:
# Creating the feature group for the danish holidays
danish_holidays_fg = fs.get_or_create_feature_group(
    name="danish_holidays",
    version=1,
    description="Danish holidays calendar.",
    online_enabled=True,
    primary_key=["date"],
)

In [ ]:
# Inserting the calender_df into the feature group named danish_holidays_fg
danish_holidays_fg.insert(calender_df)

In [ ]:
danish_holidays_feature_descriptions = [
    {"name": "date", "description": "Date in the calendar"},
    {"name": "type", "description": "Holyday or not holyday"},
]

for desc in danish_holidays_feature_descriptions: 
    danish_holidays_fg.update_feature_description(desc["name"], desc["description"])

---
## <span style="color:#2656a3;">⏭️ **Next:** Part 02: Feature Pipeline </span>

In the next notebook, you will be generating new data for the Feature Groups.